In [11]:
from monty.serialization import loadfn, dumpfn
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet, MPMetalRelaxSet
from pymatgen.io.vasp import Poscar
from doped.chemical_potentials import CompetingPhases, CompetingPhasesAnalyzer
from pymatgen.entries.computed_entries import ComputedEntry
import os

In [3]:
supercell = Poscar.from_file("doped-defs-AlClO/AlOCl_bulk/vasp_std/CONTCAR")
bulk_supercell = supercell.structure

In [7]:
cp = CompetingPhases(composition='AlClO', extrinsic='Cr', energy_above_hull=0.05)

In [55]:
for entry in cp.entries:
    dir_path = os.path.join(
        "AlClO",
        "AlClO_competing_phases",
        f"{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}",
        "vasp_std"
    )
    os.makedirs(dir_path, exist_ok=True)

In [56]:
calculated_entries = {}
for entry in cp.entries:
    key =  entry.data['material_id'] + '-' + entry.structure.composition.reduced_formula
    if key in calculated_entries:
        continue
    # if entry.data['summary']['band_gap'] == 0:
    else:
        entries.append(key)
        if entry.data['summary']['band_gap'] == 0:
            mp_relax_set = MPMetalRelaxSet(entry.structure)
            mp_relax_set.write_input(f"AlClO/AlClO_competing_phases/{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}/vasp_std", poscar=True)
        else:
            mp_relax_set = MPRelaxSet(entry.structure)
            mp_relax_set.write_input(f"AlClO/AlClO_competing_phases/{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}/vasp_std", poscar=True)
        calculated_entries[key] = entry

In [58]:
cpa = CompetingPhasesAnalyzer(composition='AlClO', entries="AlClO/AlClO_competing_phases/")

Parsing vaspruns...: 100%|██████████████████████████████████████████████████████████████| 22/22 [00:08<00:00,  2.64it/s]


In [60]:
cpa.unstable_host

False

In [61]:
chempots = cpa.chempots

In [72]:
formation_energies = {}
for key in chempots['limits'].keys():
    def_form_e = -337.73769164 + chempots['limits'][key]['Al'] - (-351.69390352) - chempots['limits'][key]['Cr']
    formation_energies[key] = def_form_e

In [73]:
formation_energies

{'AlClO-Cl2-Al2O3-CrCl3': 5.815879809999988,
 'AlClO-AlCl3-Cl2-CrCl3': 5.8858598100000314,
 'AlClO-Al-Al2O3-Al45Cr7': 8.297749809999997,
 'AlClO-AlCl3-Al-Al45Cr7': 8.297749809999997}